In [1]:
import os

In [2]:
pwd

'd:\\PW_DS\\Deep Learning\\End-To-End-Plant-Disease-Classification-using-CNN\\research'

In [3]:
os.chdir('..')

In [4]:
pwd

'd:\\PW_DS\\Deep Learning\\End-To-End-Plant-Disease-Classification-using-CNN'

## Entity

In [5]:
from dataclasses import  dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class PrepareCallbacks:
    root_dir: Path
    tensorboard_logs: Path
    model_checkpoint: Path

## Config Manager

In [7]:
from plant_disease_clf.utils.common import  create_directories, read_yaml
from plant_disease_clf.constants import  *

In [8]:
class ConfigManager:
    def __init__(self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_prepare_callbacks_config(self):
        self.config = self.config.callbacks

        return PrepareCallbacks(
            root_dir = self.config.root_dir,
            tensorboard_logs = self.config.tensorboard_logs,
            model_checkpoint = self.config.model_checkpoint
        )



## Components

In [9]:
import os,sys
from plant_disease_clf.logger import  logging
from plant_disease_clf.exception import  CustomException
from plant_disease_clf.utils.common import  create_directories

import tensorflow as tf


In [10]:
class Callbacks:
    def __init__(self, config: PrepareCallbacks):
        self.config = config

    def get_tb_callback(self):
        return tf.keras.callbacks.TensorBoard(log_dir=self.config.tensorboard_logs, histogram_freq=1)

    def get_model_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.model_checkpoint, save_best_only=True)

    def get_callbacks_list(self):
        return [
            self.get_tb_callback(),
            self.get_model_ckpt_callback()
        ]
    

# Pipeline

In [11]:
from plant_disease_clf.logger import  logging
from plant_disease_clf.exception import  CustomException

try:
    config_manager = ConfigManager()
    prepare_callbacks = config_manager.get_prepare_callbacks_config()
    callbacks = Callbacks(prepare_callbacks)
    callbacks_list = callbacks.get_callbacks_list()
    logging.info("Callbacks are created successfully")

except Exception as e:
    raise CustomException(e,sys)